In [1]:
from google.colab import drive
from os.path import join 

ROOT = "/content/gdrive/"
drive.mount(ROOT)

Mounted at /content/gdrive/


## 손글씨 데이터에 대해 rawdata 및 전처리, augmentation 수행한 최종 데이터 얻음

#### 손글씨 데이터 파악

In [5]:
import json
def getListOfHandWritingFileNames(listOfFemaleHandWritingFileNames, listOfMaleHandWritingFileNames):
    handwritingMetaFile_location = '/content/gdrive/Shareddrives/machine_learning_in_practice/한국어 글자체 이미지/01.손글씨/handwriting_data_info1.json'

    with open(handwritingMetaFile_location, 'r') as f:
        hw_json_data = json.load(f)

    hw_ClassToFileNames, hw_WordsToFileNames= {}, {}
    hw_FileNameToClass = {}
    hw_syllableDataLength = 0

    for idx, img_data in enumerate(hw_json_data['images']):
        # idx 번째 이미지 데이터들에 대해 {클래스 : [파일명, 파일명, ...]}
        class_ = hw_json_data['annotations'][idx]['text']
        fileName = hw_json_data['images'][idx]['file_name']
        if len(class_) == 1:    
            # 한 글자인 경우만 ClassToFileNames에 넣기
            if class_ not in hw_ClassToFileNames.keys(): # idx번째 이미지의 클래스가 새로운 클래스이면
                hw_ClassToFileNames[class_] = [] # 빈 리스트 생성
            hw_ClassToFileNames[class_].append(fileName)
            hw_syllableDataLength += 1
            if hw_json_data['annotations'][1]['attributes']['gender'] == '여':
                listOfFemaleHandWritingFileNames.append(fileName)
            else:
                listOfMaleHandWritingFileNames.append(fileName)
        else:
            # 단어, 문장의 경우 WordsToFileNames에 넣기
            if class_ not in hw_WordsToFileNames.keys(): # idx번째 이미지의 클래스가 새로운 클래스이면
                hw_WordsToFileNames[class_] = [] # 빈 리스트 생성
            hw_WordsToFileNames[class_].append(fileName)
    
    for idx, img_data in enumerate(hw_json_data['images']):
        fileName = hw_json_data['images'][idx]['file_name']
        class_ = hw_json_data['annotations'][idx]['text']
        hw_FileNameToClass[fileName] = class_
    
    print("*** Handwriting Data Description ***")
    print("\t전체 데이터 수 : ", len(hw_json_data['images']) )# 여기에는 문장 데이터, 단어 데이터, 음절 데이터 모두 포함됨
    print("\t음절 클래스 수 : ", len(hw_ClassToFileNames) ) # 한국어 글자 종류는 11,172가지.
    print("\t단어/문장 클래스 수: ", len(hw_WordsToFileNames))
    print("\t음절 데이터 수 : ", hw_syllableDataLength)
    print("\t단어/문장 데이터 수 : ", len(hw_json_data['images']) - hw_syllableDataLength)
    print(f"\t한 음절 당 평균 {hw_syllableDataLength/len(hw_ClassToFileNames)} 개 손글씨 데이터 존재함")

    return hw_ClassToFileNames, hw_FileNameToClass

In [6]:
listOfFemaleHandWritingFileNames, listOfMaleHandWritingFileNames = [], []

hw_ClassToFileNames, hw_FileNameToClass = getListOfHandWritingFileNames(listOfFemaleHandWritingFileNames, listOfMaleHandWritingFileNames)

print("여성 필기체 data 수 : ", len(listOfFemaleHandWritingFileNames))
print("남성 필기체 data 수 : ", len(listOfMaleHandWritingFileNames))

*** Handwriting Data Description ***
	전체 데이터 수 :  1170622
	음절 클래스 수 :  11172
	단어/문장 클래스 수:  6054
	음절 데이터 수 :  761109
	단어/문장 데이터 수 :  409513
	한 음절 당 평균 68.1264769065521 개 손글씨 데이터 존재함
여성 필기체 data 수 :  761109
남성 필기체 data 수 :  0


#### newClassToFileName 

In [2]:
import csv 
path = '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/pretrain/classes.csv'
f = open(path,'r')
rdr = csv.reader(f)
for line in rdr:
    classes = line 
f.close()

newClassToFileNames = {} # key 1268개
for syllable in classes:
    newClassToFileNames[syllable] = hw_ClassToFileNames[syllable]

1268

#### FileNameToClass.pickle - key를 한정 (기존 FileNameToClass는 11172개의 class 모두 담고 있음 => 1268개 클래스로 축소)

In [9]:
classes = newClassToFileNames.keys()
new_hwFileNameToClass = dict()
for fileName in hw_FileNameToClass.keys():
    if hw_FileNameToClass[fileName] not in classes:
        continue
    new_hwFileNameToClass[fileName] = hw_FileNameToClass[fileName]

print("클래스 축소 전 데이터 수 : ", len(hw_FileNameToClass))
print("클래스 축소 후 데이터 수 : ", len(new_hwFileNameToClass))

클래스 축소 전 데이터 수 :  1170622
클래스 축소 후 데이터 수 :  106076


#### raw_handwritten.zip | 1268개 클래스에 해당하는 이미지 저장

In [10]:
!unzip '/content/gdrive/Shareddrives/machine_learning_in_practice/한국어 글자체 이미지/01.손글씨/01_handwriting_syllable_images.zip' -d .

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./1_syllable/00165297.png  
  inflating: ./1_syllable/00165304.png  
  inflating: ./1_syllable/00165305.png  
  inflating: ./1_syllable/00165306.png  
  inflating: ./1_syllable/00165311.png  
  inflating: ./1_syllable/00165313.png  
  inflating: ./1_syllable/00165319.png  
  inflating: ./1_syllable/00165327.png  
  inflating: ./1_syllable/00165341.png  
  inflating: ./1_syllable/00165343.png  
  inflating: ./1_syllable/00165345.png  
  inflating: ./1_syllable/00165351.png  
  inflating: ./1_syllable/00165357.png  
  inflating: ./1_syllable/00165359.png  
  inflating: ./1_syllable/00165360.png  
  inflating: ./1_syllable/00165362.png  
  inflating: ./1_syllable/00165363.png  
  inflating: ./1_syllable/00165376.png  
  inflating: ./1_syllable/00165381.png  
  inflating: ./1_syllable/00165385.png  
  inflating: ./1_syllable/00165387.png  
  inflating: ./1_syllable/00165400.png  
  inflating: ./1_syllable/00165408.png  
  inflating: ./1_syll

In [11]:
import os

path = '/content/1_syllable'
file_list = os.listdir(path)

# new_pFileNameToClass 는 1268개 클래스에 해당하는 image filename 들만 key 로 있음
newFileList = []
newClassList = []
for filename in file_list:
    try: # new_pFileNameToClass 이용함 key 에러 뜨면 넘겨야 함
        class_ = new_hwFileNameToClass[filename]
        newFileList.append(filename)
        newClassList.append(class_)
    except KeyError:
        # print(filename)
        pass
print(len(newFileList))
print(len(newClassList))

15323
15323


In [13]:
import csv

filepath = '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/transfer/fileName.csv'
with open(filepath, 'w', newline='') as f: 
    writer = csv.writer(f) 
    writer.writerow(newFileList)

In [15]:
import os
import zipfile

new_zips= zipfile.ZipFile('raw_handwritten.zip', 'w') # 
num_zipfile = 0

for folder, subfolders, files in os.walk('/content/1_syllable'): 
    for file in files:
        try:
            new_hwFileNameToClass[file] # 1268 클래스에 속하는 데이터만 zip함  총 112300개
            new_zips.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './'), compress_type = zipfile.ZIP_DEFLATED)
            num_zipfile += 1
        except KeyError:
            pass
new_zips.close()
print(num_zipfile)

15323


In [16]:
import shutil

savePath = '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/transfer'

shutil.move('/content/raw_handwritten.zip', savePath) 

'/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/transfer/raw_handwritten.zip'

#### croppedImg 생성

In [17]:
!unzip '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/transfer/raw_handwritten.zip' -d ./rawdata

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./rawdata/1_syllable/00019182.png  
  inflating: ./rawdata/1_syllable/00102850.png  
  inflating: ./rawdata/1_syllable/00047163.png  
  inflating: ./rawdata/1_syllable/00188659.png  
  inflating: ./rawdata/1_syllable/00177477.png  
  inflating: ./rawdata/1_syllable/00167173.png  
  inflating: ./rawdata/1_syllable/00115980.png  
  inflating: ./rawdata/1_syllable/00133629.png  
  inflating: ./rawdata/1_syllable/00025226.png  
  inflating: ./rawdata/1_syllable/00188469.png  
  inflating: ./rawdata/1_syllable/00033720.png  
  inflating: ./rawdata/1_syllable/00117524.png  
  inflating: ./rawdata/1_syllable/00148896.png  
  inflating: ./rawdata/1_syllable/00094249.png  
  inflating: ./rawdata/1_syllable/00117695.png  
  inflating: ./rawdata/1_syllable/00018883.png  
  inflating: ./rawdata/1_syllable/00114856.png  
  inflating: ./rawdata/1_syllable/00172563.png  
  inflating: ./rawdata/1_syllable/00134049.png  
  inflating: ./rawdata/1_syllable

In [19]:
import cv2
import csv
import os
import numpy as np
from PIL import Image, ImageOps
import matplotlib.image as img 

savePath = '/content/preprocessed' 

path = '/content/rawdata/1_syllable'
file_list_ = os.listdir(path)

for filename in file_list_:
    filePath = path + '/' + filename
    image=Image.open(filePath)
    image.load()
    imageSize = image.size
    
    # remove alpha channel
    invert_im = image.convert("RGB") 
    
    # invert image (so that white is 0)
    invert_im = ImageOps.invert(invert_im)
    imageBox = invert_im.getbbox()
    
    cropped=image.crop(imageBox)
    print(filePath, "Size:", imageSize, "New Size:", imageBox)
    cropped.save(savePath + '/' + filename)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/rawdata/1_syllable/00019182.png Size: (174, 195) New Size: (0, 0, 174, 195)
/content/rawdata/1_syllable/00102850.png Size: (175, 194) New Size: (0, 0, 175, 194)
/content/rawdata/1_syllable/00047163.png Size: (174, 195) New Size: (0, 0, 174, 195)
/content/rawdata/1_syllable/00188659.png Size: (171, 195) New Size: (0, 0, 171, 195)
/content/rawdata/1_syllable/00177477.png Size: (172, 194) New Size: (0, 0, 172, 194)
/content/rawdata/1_syllable/00167173.png Size: (172, 194) New Size: (0, 0, 172, 194)
/content/rawdata/1_syllable/00115980.png Size: (172, 194) New Size: (0, 0, 172, 194)
/content/rawdata/1_syllable/00133629.png Size: (173, 194) New Size: (0, 0, 173, 194)
/content/rawdata/1_syllable/00025226.png Size: (174, 194) New Size: (0, 0, 174, 194)
/content/rawdata/1_syllable/00188469.png Size: (167, 195) New Size: (0, 0, 167, 195)
/content/rawdata/1_syllable/00033720.png Size: (171, 193) New Size: (0, 0, 171, 193)
/content/rawdata/1_syllable/0

In [20]:
import os
import zipfile

new_zips= zipfile.ZipFile('preprocessed.zip', 'w')
 
for folder, subfolders, files in os.walk('/content/preprocessed/'): 
    for file in files:
        new_zips.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './'), compress_type = zipfile.ZIP_DEFLATED)

new_zips.close()

In [21]:
import shutil

savePath = '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/transfer'

shutil.move('preprocessed.zip', savePath)

'/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/transfer/preprocessed.zip'

## github 커밋

In [22]:
MY_GOOGLE_DRIVE_PATH = "/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital"
%cd "{MY_GOOGLE_DRIVE_PATH}"

/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital


In [ ]:
!git config --global user.email dkwjd0824@khu.ac.kr  # 이메일 입력 ex) qhrqufdlek@naver.com
!git config --global user.name  hyeneung #깃헙 아이디 입력 ex)luckydipper
!git pull

In [ ]:
!git status

In [ ]:
!git add GAN/PrintedDataset.ipynb

In [ ]:
!git commit -m"[ADD] Reload printedDataset"

[main dbfc005] [ADD] Reload printedDataset
 1 file changed, 1 insertion(+)
 create mode 100644 GAN/PrintedDataset.ipynb


In [ ]:
!git push

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 110.88 KiB | 3.82 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/PILGI-Analog-To-Digital/Analog-PILGI-to-DIgital.git
   d9a82c8..dbfc005  main -> main
